In [ ]:

import numpy as np
import time
import matplotlib.pyplot as plt
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from qr import svd_from_eigen
from bfgs import bfgs, l_bfgs
from adam import adam

Eigenvalues calculated by our QR algorithm: [np.float64(-0.06870782332995608), np.float64(2.722245633762534), np.float64(5.346462189567423)]
Eigenvalues calculated by NumPy: [np.float64(-0.0687078233299574), np.float64(2.722245633762532), np.float64(5.346462189567425)]

Comparison:
Are the results close? True
U:
 [[ 0.2298477   0.88346102  0.        ]
 [ 0.52474482  0.24078249  0.        ]
 [ 0.81964194 -0.40189603  0.        ]]

Σ:
 [[9.52551809 0.        ]
 [0.         0.51430058]
 [0.         0.        ]]

V^T:
 [[ 0.61962948  0.78489445]
 [-0.78489445  0.61962948]]

A ≈ U Σ V^T:
 [[1. 2.]
 [3. 4.]
 [5. 6.]]
Our U:
 [[ 0.2298477   0.88346102  0.        ]
 [ 0.52474482  0.24078249  0.        ]
 [ 0.81964194 -0.40189603  0.        ]]

NumPy U:
 [[-0.2298477   0.88346102  0.40824829]
 [-0.52474482  0.24078249 -0.81649658]
 [-0.81964194 -0.40189603  0.40824829]]

Our Sigma:
 [[9.52551809 0.        ]
 [0.         0.51430058]
 [0.         0.        ]]

NumPy Sigma:
 [[9.52551809 0.       

In [2]:
# ===== Вспомогательные функции =====
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def add_bias(X):
    return np.hstack([X, np.ones((X.shape[0], 1))])

def finite_diff_grad(f, w, h=1e-6):
    g = np.zeros_like(w)
    for i in range(len(w)):
        e = np.zeros_like(w)
        e[i] = 1.0
        g[i] = (f(w + h * e) - f(w - h * e)) / (2 * h)
    return g

In [3]:


# ===== Подготовка данных =====
wine = load_wine()
X, y = wine.data, wine.target

# Берём только 2 класса, чтобы получить бинарную задачу
mask = y < 2
X, y = X[mask], y[mask]

# Нормализация
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# PCA: 2 компоненты (→ всего 3 признака с bias)
U, S, V = svd_from_eigen(X_scaled)
X_pca = X_scaled @ V[:, :2]
X_pca = add_bias(X_pca)

X_train, X_test, y_train, y_test = train_test_split(
    X_pca, y, test_size=0.3, random_state=42, stratify=y
)


In [4]:

# ===== Модель =====
class LogisticRegressionBinary:
    def __init__(self, X, y, reg=1e-2):
        self.X = X
        self.y = y
        self.n, self.d = X.shape
        self.reg = reg

    def loss(self, w):
        z = self.X @ w
        p = sigmoid(z)
        loss = -np.mean(self.y * np.log(p + 1e-12) + (1 - self.y) * np.log(1 - p + 1e-12))
        reg_term = 0.5 * self.reg * np.sum(w[:-1] ** 2)
        return loss + reg_term

    def accuracy(self, w, X_test, y_test):
        preds = sigmoid(X_test @ w) >= 0.5
        return np.mean(preds == y_test)

In [5]:
# ===== Инициализация модели =====
model = LogisticRegressionBinary(X_train, y_train)
f_loss = model.loss

# Обёртка, совместимая с сигнатурой bfgs.py
def f_loss_wrapped(*args):
    w = np.array(args)
    return f_loss(w)

def grad_fd_wrapped(*args):
    w = np.array(args)
    return finite_diff_grad(f_loss, w, h=1e-5)

In [6]:

# ====== Отслеживание динамики ======
loss_history_bfgs = []
loss_history_lbfgs = []
loss_history_adam = []

def f_loss_track_bfgs(*args):
    w = np.array(args)
    val = f_loss(w)
    loss_history_bfgs.append(val)
    return val

def f_loss_track_lbfgs(*args):
    w = np.array(args)
    val = f_loss(w)
    loss_history_lbfgs.append(val)
    return val

def f_loss_track_adam(*args):
    w = np.array(args)
    val = f_loss(w)
    loss_history_adam.append(val)
    return val


In [7]:



# ===== Запуск оптимизаторов =====
results = {}

# --- BFGS ---
start = time.time()
res_bfgs = bfgs(f_loss_track_bfgs, grad_fd_wrapped, eps=1e-5)
t_bfgs = time.time() - start
results["BFGS"] = {
    "x": np.array(res_bfgs),
    "acc": model.accuracy(np.array(res_bfgs), X_test, y_test),
    "time": t_bfgs,
}

# --- L-BFGS ---
start = time.time()
res_lbfgs = l_bfgs(f_loss_track_lbfgs, grad_fd_wrapped, eps=1e-5, m=7)
t_lbfgs = time.time() - start
results["L-BFGS"] = {
    "x": np.array(res_lbfgs),
    "acc": model.accuracy(np.array(res_lbfgs), X_test, y_test),
    "time": t_lbfgs,
}

# --- Adam ---
start = time.time()
res_adam, loss_history_adam = adam(f_loss_track_adam, grad_fd_wrapped)
t_adam = time.time() - start

results["Adam"] = {
    "x": np.array(res_adam),
    "acc": model.accuracy(np.array(res_adam), X_test, y_test),
    "time": t_adam,
}





C:\Users\roma\AppData\Local\Temp\ipykernel_10608\3285654191.py:3: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-z))


In [ ]:

print("\n=== Сравнение оптимизаторов ===")
print("{:<10} {:>12} {:>12}".format("Метод", "Точность", "Время(с)"))
for name, res in results.items():
    print(f"{name:<10} {res['acc']:>12.4f} {res['time']:>12.3f}")





=== Сравнение оптимизаторов ===
Метод          Точность     Время(с)
BFGS             0.9231        0.337
L-BFGS           0.9231        0.009
Adam             0.9231        0.535
